<a href="https://colab.research.google.com/github/ZacharySoo01/I320D_AppliedML_Project/blob/main/Applied_ML_Final_Project_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pickle

# Load the saved model
with open('MLPRmodel.pkl', 'rb') as file:
    model = pickle.load(file)


In [ ]:
print(type(model))

<class 'sklearn.neural_network._multilayer_perceptron.MLPRegressor'>


In [ ]:
!pip install gradio

In [ ]:
import numpy as np
import pandas as pd

# Load model and scaler
feature_artifact_file = open("scaler.pkl", "rb")
pca_artifact_file = open("pca.pkl", "rb")
feature_scaler = pickle.load(feature_artifact_file)
pca_obj = pickle.load(pca_artifact_file)
dataset = pd.read_csv("new_train.csv")

def execute_inference_pipeline(state, county, first_day_of_month, active, pct_bb_2017, pct_bb_2018, pct_bb_2019, pct_bb_2020, pct_bb_2021, pct_college_2017, pct_college_2018, pct_college_2019, pct_college_2020,\
                               pct_college_2021, pct_foreign_born_2017, pct_foreign_born_2018, pct_foreign_born_2019, pct_foreign_born_2020, pct_foreign_born_2021, pct_it_workers_2017,\
                               pct_it_workers_2018, pct_it_workers_2019, pct_it_workers_2020, pct_it_workers_2021, median_hh_inc_2017, median_hh_inc_2018, median_hh_inc_2019, median_hh_inc_2020,\
                               median_hh_inc_2021):
    # transform categorical features
    county_target_map = dataset.groupby('county')['microbusiness_density'].mean().to_dict()
    county = county_target_map.get(county, None)
    state_target_map = dataset.groupby('state')['microbusiness_density'].mean().to_dict()
    state = state_target_map.get(state, None)
    date_target_map = dataset.groupby('first_day_of_month')['microbusiness_density'].mean().to_dict()
    first_day_of_month = date_target_map.get(first_day_of_month, None)
    cat_features = np.array([[county, state, first_day_of_month]])

    # transform numerical features
    features = np.array([[active, pct_bb_2017, pct_bb_2018, pct_bb_2019, pct_bb_2020, pct_bb_2021, pct_college_2017, pct_college_2018, pct_college_2019, pct_college_2020,\
                               pct_college_2021, pct_foreign_born_2017, pct_foreign_born_2018, pct_foreign_born_2019, pct_foreign_born_2020, pct_foreign_born_2021, pct_it_workers_2017,\
                               pct_it_workers_2018, pct_it_workers_2019, pct_it_workers_2020, pct_it_workers_2021, median_hh_inc_2017, median_hh_inc_2018, median_hh_inc_2019, median_hh_inc_2020,\
                               median_hh_inc_2021]])
    scaled_features = feature_scaler.transform(features)

    combined_features = np.concatenate((cat_features, scaled_features), axis=1)

    # transform with pca
    features_transformed = pca_obj.transform(combined_features)
    # predict with model
    prediction = model.predict(features_transformed)

    return prediction[0]

In [ ]:
# Create input and output components for Gradio
import gradio as gr

states = list(dataset["state"].unique())
states.sort()
counties = list(dataset["county"].unique())
counties.sort()
first_day_of_month = list(dataset["first_day_of_month"].unique())
first_day_of_month.sort()
inputs = [
    gr.Dropdown(
        choices=states,
        label="Choose a State",
        value="Texas"
    ),
    gr.Dropdown(
        choices=counties,
        label="Choose a County",
        value="Travis County"
    ),
     gr.Dropdown(
        choices=first_day_of_month,
        label="Choose the first day of month of the business",
        value="2021-01-01"
    ),
    gr.Number(label="Enter # of active microbusinesses in the county", value=1),
    gr.Number(label="Enter % of households in the county with access to broadband of any type (2017)", value=1),
    gr.Number(label="Enter % of households in the county with access to broadband of any type (2018)", value=1),
    gr.Number(label="Enter % of households in the county with access to broadband of any type (2019)", value=1),
    gr.Number(label="Enter % of households in the county with access to broadband of any type (2020)", value=1),
    gr.Number(label="Enter % of households in the county with access to broadband of any type (2021)", value=1),
    gr.Number(label="Enter % of the population in the county over age 25 with a 4-year college degree (2017)", value=1),
    gr.Number(label="Enter % of the population in the county over age 25 with a 4-year college degree (2018)", value=1),
    gr.Number(label="Enter % of the population in the county over age 25 with a 4-year college degree (2019)", value=1),
    gr.Number(label="Enter % of the population in the county over age 25 with a 4-year college degree (2020)", value=1),
    gr.Number(label="Enter % of the population in the county over age 25 with a 4-year college degree (2021)", value=1),
    gr.Number(label="Enter % of the population in the county born outside of the United States (2017)", value=1),
    gr.Number(label="Enter % of the population in the county born outside of the United States (2018)", value=1),
    gr.Number(label="Enter % of the population in the county born outside of the United States (2019)", value=1),
    gr.Number(label="Enter % of the population in the county born outside of the United States (2020)", value=1),
    gr.Number(label="Enter % of the population in the county born outside of the United States (2021)", value=1),
    gr.Number(label="Enter % of the workforce in the county employed in IT related industries (2017)", value=1),
    gr.Number(label="Enter % of the workforce in the county employed in IT related industries (2018)", value=1),
    gr.Number(label="Enter % of the workforce in the county employed in IT related industries (2019)", value=1),
    gr.Number(label="Enter % of the workforce in the county employed in IT related industries (2020)", value=1),
    gr.Number(label="Enter % of the workforce in the county employed in IT related industries (2021)", value=1),
    gr.Number(label="Enter Median household income in the county (2017)", value=1),
    gr.Number(label="Enter Median household income in the county (2018)", value=1),
    gr.Number(label="Enter Median household income in the county (2019)", value=1),
    gr.Number(label="Enter Median household income in the county (2020)", value=1),
    gr.Number(label="Enter Median household income in the county (2021)", value=1)
]

output = gr.Textbox(label="Prediction")

In [ ]:
dataset.describe()

,Unnamed: 0,cfips,microbusiness_density,active,pct_bb_2017,pct_bb_2018,pct_bb_2019,pct_bb_2020,pct_bb_2021,pct_college_2017,...,pct_it_workers_2017,pct_it_workers_2018,pct_it_workers_2019,pct_it_workers_2020,pct_it_workers_2021,median_hh_inc_2017,median_hh_inc_2018,median_hh_inc_2019,median_hh_inc_2020,median_hh_inc_2021
count,20900.000000,20900.000000,20900.000000,2.090000e+04,20900.000000,20900.000000,20900.000000,20900.00000,20900.000000,20900.000000,...,20900.000000,20900.000000,20900.000000,20900.000000,20900.000000,20900.000000,20900.000000,20900.000000,20900.000000,20900.000000
mean,60564.565789,30127.718182,3.601077,6.365900e+03,69.859096,72.623756,75.350349,78.50567,80.502612,13.744880,...,1.424249,1.383818,1.338206,1.310445,1.276813,49688.472727,51501.432010,53407.729282,54951.801244,58172.731148
std,35216.717461,15135.570847,3.149898,3.290948e+04,9.689613,9.256553,8.829965,8.21027,7.841947,5.485368,...,1.011134,0.963243,0.941912,1.003357,0.983192,13108.829048,13657.792662,14148.501509,14581.947919,15482.906849
min,0.000000,1001.000000,0.000000,0.000000e+00,24.500000,25.700000,34.800000,33.30000,37.000000,2.400000,...,0.000000,0.000000,0.000000,0.000000,0.000000,19264.000000,20188.000000,21504.000000,22292.000000,17109.000000
25%,30314.500000,18165.000000,1.632008,1.410000e+02,64.200000,67.400000,70.500000,74.10000,76.300000,9.700000,...,0.800000,0.800000,0.700000,0.700000,0.600000,41103.000000,42348.000000,44100.000000,45588.000000,48142.000000
50%,60062.000000,29119.000000,2.563864,4.750000e+02,70.800000,73.600000,76.500000,79.60000,81.700000,12.800000,...,1.300000,1.300000,1.200000,1.200000,1.100000,47992.000000,49766.000000,51726.000000,52816.000000,55904.000000
75%,90963.750000,45039.000000,4.448178,2.056000e+03,76.400000,78.700000,81.400000,84.10000,85.900000,16.700000,...,1.900000,1.800000,1.800000,1.700000,1.700000,55649.000000,57479.000000,59725.000000,61391.250000,64926.000000
max,122263.000000,56045.000000,24.638775,1.153292e+06,94.600000,95.500000,96.000000,97.10000,97.600000,39.000000,...,17.400000,11.700000,10.500000,15.200000,15.200000,129588.000000,136268.000000,142299.000000,147111.000000,156821.000000


In [ ]:
gr.Interface(
    fn=execute_inference_pipeline,
    inputs=inputs,
    outputs=output,
    title="Microbusiness Density Prediction",
    description="Enter information to predict microbusiness density for a given location and date."
).launch(debug = True)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://5c0cda190654e9fc17.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but PCA was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but PCA was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but PCA was fitted with fe